## Noor Sharma Capstone Project IBM

In [ ]:
**Importing all the required libraries**

In [299]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
#=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [118]:
website_url = requests.get("https://en.wikipedia.org/wiki/Regional_Municipality_of_Durham").text

In [119]:
My_table = soup.find('table',{'class':'wikitable'})

My_table

<table class="wikitable" style="text-align:left;">
<tbody><tr>
<th>Municipality</th>
<th>2016 Population<sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup>
</th></tr>
<tr>
<td>City of <a href="/wiki/Oshawa" title="Oshawa">Oshawa</a></td>
<td>159,458
</td></tr>
<tr>
<td>Town of <a href="/wiki/Whitby,_Ontario" title="Whitby, Ontario">Whitby</a></td>
<td>128,377
</td></tr>
<tr>
<td>Town of <a href="/wiki/Ajax,_Ontario" title="Ajax, Ontario">Ajax</a></td>
<td>119,677
</td></tr>
<tr>
<td>Municipality of <a href="/wiki/Clarington" title="Clarington">Clarington</a></td>
<td>92,013
</td></tr>
<tr>
<td>City of <a href="/wiki/Pickering,_Ontario" title="Pickering, Ontario">Pickering</a></td>
<td>91,771
</td></tr>
<tr>
<td>Township of <a href="/wiki/Scugog" title="Scugog">Scugog</a></td>
<td>21,617
</td></tr>
<tr>
<td>Township of <a href="/wiki/Uxbridge,_Ontario" title="Uxbridge, Ontario">Uxbridge</a></td>
<td>21,176
</td></tr>
<tr>
<td>Township of <a href="/wiki/Brock,_Ont

In [120]:
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Regional Municipality of Durham - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Regional_Municipality_of_Durham","wgTitle":"Regional Municipality of Durham","wgCurRevisionId":915482479,"wgRevisionId":915482479,"wgArticleId":254159,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with reference errors","Pages with duplicate reference names","Articles with short description","Coordinates on Wikidata","Articles to be expanded from June 2008","All articles to be expanded","Articles using small message boxes","All articles with unsourced statements","Articles with unsourced statements from March 2015","Commons

In [121]:
links = My_table.findAll('a')
links

[<a href="#cite_note-2">[2]</a>,
 <a href="/wiki/Oshawa" title="Oshawa">Oshawa</a>,
 <a href="/wiki/Whitby,_Ontario" title="Whitby, Ontario">Whitby</a>,
 <a href="/wiki/Ajax,_Ontario" title="Ajax, Ontario">Ajax</a>,
 <a href="/wiki/Clarington" title="Clarington">Clarington</a>,
 <a href="/wiki/Pickering,_Ontario" title="Pickering, Ontario">Pickering</a>,
 <a href="/wiki/Scugog" title="Scugog">Scugog</a>,
 <a href="/wiki/Uxbridge,_Ontario" title="Uxbridge, Ontario">Uxbridge</a>,
 <a href="/wiki/Brock,_Ontario" title="Brock, Ontario">Brock</a>]

In [207]:
Durham = []
for link in links:
    Durham.append(link.get('title'))
    
print(Durham)

del Durham[0:2]

Durham

[None, 'Oshawa', 'Whitby, Ontario', 'Ajax, Ontario', 'Clarington', 'Pickering, Ontario', 'Scugog', 'Uxbridge, Ontario', 'Brock, Ontario']


['Whitby, Ontario',
 'Ajax, Ontario',
 'Clarington',
 'Pickering, Ontario',
 'Scugog',
 'Uxbridge, Ontario',
 'Brock, Ontario']

In [208]:
df = pd.DataFrame()
df['Durham'] = Durham
df

,Durham
0,"Whitby, Ontario"
1,"Ajax, Ontario"
2,Clarington
3,"Pickering, Ontario"
4,Scugog
5,"Uxbridge, Ontario"
6,"Brock, Ontario"


In [211]:
def get_coords_local(Durham, output_as='center'):
    """
    get the bounding box of a locality in WGS84 given its name

    Parameters
    ----------
    Durham : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]

    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             Durham+', Durham Region, Ontario, Canada',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output

In [212]:
df2 = df.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(Durham=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln

df2.shape

Whitby, Ontario
Ajax, Ontario
Clarington
Pickering, Ontario
Scugog
Uxbridge, Ontario
Brock, Ontario


(7, 3)

In [213]:
df2

,Durham,Latitude,Longitude
0,"Whitby, Ontario",43.899871,-78.940201
1,"Ajax, Ontario",43.850374,-79.024658
2,Clarington,43.968686,-78.651357
3,"Pickering, Ontario",43.835765,-79.090576
4,Scugog,44.115055,-78.905989
5,"Uxbridge, Ontario",44.107556,-79.127655
6,"Brock, Ontario",44.350079,-79.101226


In [214]:
address = 'Durham Region, Ontario, Canada'
geolocator = Nominatim(user_agent="capstoneProject")
location = geolocator.geocode(address, timeout=60, exactly_one=True)
latitude = location.latitude
longitude = location.longitude
print('The decimal coordinates of York region are {}, {}.'.format(latitude, longitude))

The decimal coordinates of York region are 44.0310471, -78.949678.


In [215]:
# make a copy of the dataframe to get it simply back if needed
df3 = df2.copy()
df3

,Durham,Latitude,Longitude
0,"Whitby, Ontario",43.899871,-78.940201
1,"Ajax, Ontario",43.850374,-79.024658
2,Clarington,43.968686,-78.651357
3,"Pickering, Ontario",43.835765,-79.090576
4,Scugog,44.115055,-78.905989
5,"Uxbridge, Ontario",44.107556,-79.127655
6,"Brock, Ontario",44.350079,-79.101226


In [218]:
map_Durham = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, local in zip(df3['Latitude'], df3['Longitude'], df3['Durham']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Durham)  
    
map_Durham

In [219]:
  #set variables
radius=5000
LIMIT=500
CLIENT_ID = 'REBHNNAYHW4E0YKM13LNF4GBKB1Q5EBJYQW50PZBQWFDIL2K' # your Foursquare ID
CLIENT_SECRET = 'SUXWYLIW3DGOKO5KAS15P1JKDHLOLIJLZVPXFFI4LXX0CORS' # your Foursquare Secret
VERSION = '20190709' # Foursquare API version
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Durham', 
                  'Durham Latitude', 
                  'Durham Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [223]:
# Use category id 4bf58dd8d48988d16c941735 to only get the burger joints
Durham_venues_fastfood = getNearbyVenues(names=df3['Durham'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d16e941735')
Durham_venues_fastfood

,Durham,Durham Latitude,Durham Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Whitby, Ontario",43.899871,-78.940201,Subway,43.900628,-78.941704,Sandwich Place
1,"Whitby, Ontario",43.899871,-78.940201,Subway,43.905661,-78.947007,Sandwich Place
2,"Ajax, Ontario",43.850374,-79.024658,Subway,43.845684,-79.020164,Sandwich Place
3,"Ajax, Ontario",43.850374,-79.024658,DQ Grill & Chill Restaurant,43.843049,-79.027976,Fast Food Restaurant
4,"Ajax, Ontario",43.850374,-79.024658,A&W Canada,43.844833,-79.019288,Fast Food Restaurant
5,"Ajax, Ontario",43.850374,-79.024658,Burger King,43.860403,-79.026786,Fast Food Restaurant
6,"Ajax, Ontario",43.850374,-79.024658,McDonald's,43.842542,-79.026541,Fast Food Restaurant
7,"Ajax, Ontario",43.850374,-79.024658,Wendy's,43.843722,-79.021461,Fast Food Restaurant
8,"Ajax, Ontario",43.850374,-79.024658,Burger King,43.841401,-79.027200,Fast Food Restaurant
9,"Ajax, Ontario",43.850374,-79.024658,Subway,43.860282,-79.026275,Sandwich Place


In [226]:
Durham_venues_fastfood.shape

(32, 7)

In [227]:
#unction to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Durham'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [232]:
Map_Durham_venues_fastfood = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(Durham_venues_fastfood, 'red', Map_Durham_venues_fastfood)
Map_Durham_venues_fastfood

In [234]:
Durham_venues_highschools = getNearbyVenues(names=df3['Durham'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13b941735')
Durham_venues_highschools.head()

,Durham,Durham Latitude,Durham Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Whitby, Ontario",43.899871,-78.940201,Ormiston Public school,43.909117,-78.947190,School
1,"Whitby, Ontario",43.899871,-78.940201,St Bernards,43.904265,-78.944320,School
2,"Whitby, Ontario",43.899871,-78.940201,Glen Dhu Public School,43.904728,-78.934425,School
3,"Whitby, Ontario",43.899871,-78.940201,Pringle Creek Public School,43.892770,-78.929562,School
4,"Ajax, Ontario",43.850374,-79.024658,St. John Ambulance,43.845220,-79.020116,School


In [235]:
York_venues_highschools.shape

(27, 7)

In [237]:
map_Durham_venues_highschools = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(Durham_venues_highschools, 'green', map_Durham_venues_highschools)
map_Durham_venues_highschools

In [240]:
Durham_venues_uni = getNearbyVenues(names=df3['Durham'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4d4b7105d754a06372d81259')
Durham_venues_uni.head()

,Durham,Durham Latitude,Durham Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Whitby, Ontario",43.899871,-78.940201,Liaison College,43.899781,-78.938804,College Administrative Building
1,"Whitby, Ontario",43.899871,-78.940201,St. Matthew the Evangelist Catholic School,43.910332,-78.949243,Community College
2,"Ajax, Ontario",43.850374,-79.024658,holy trinity school,43.857315,-79.031967,College Arts Building
3,"Ajax, Ontario",43.850374,-79.024658,Dr. Roberta Bondar Public School,43.851363,-79.039564,College Gym
4,"Ajax, Ontario",43.850374,-79.024658,L'entenoir,43.859618,-79.026140,College Cafeteria


In [242]:
Durham_venues_uni.shape

(13, 7)

In [243]:
map_Durham_venues_uni= folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(Durham_venues_uni, 'green', map_Durham_venues_uni)
map_Durham_venues_uni

In [244]:
Durham_venues_office = getNearbyVenues(names=df3['Durham'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4d4b7105d754a06375d81259')
Durham_venues_office 

,Durham,Durham Latitude,Durham Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Whitby, Ontario",43.899871,-78.940201,Family Care Medical Centre,43.900362,-78.941539,Doctor's Office
1,"Whitby, Ontario",43.899871,-78.940201,Regional Municipality of Durham Headquarters,43.898425,-78.940615,Office
2,"Whitby, Ontario",43.899871,-78.940201,Whitby Public Library - North,43.899825,-78.938810,Library
3,"Whitby, Ontario",43.899871,-78.940201,Business Advisory Centre Durham (BACD),43.899887,-78.940566,Non-Profit
4,"Whitby, Ontario",43.899871,-78.940201,Rossland Optical,43.899744,-78.938691,Medical Center
5,"Whitby, Ontario",43.899871,-78.940201,ServiceOntario,43.899319,-78.944229,Government Building
6,"Whitby, Ontario",43.899871,-78.940201,Whitby Family Physicians,43.899794,-78.939125,Doctor's Office
7,"Whitby, Ontario",43.899871,-78.940201,Dr. G Hattay,43.899691,-78.938941,Dentist's Office
8,"Whitby, Ontario",43.899871,-78.940201,711 rossland rd E whitby,43.899680,-78.937959,Office
9,"Whitby, Ontario",43.899871,-78.940201,Dr Office,43.900692,-78.940705,Doctor's Office


In [245]:
Durham_venues_office.shape

(156, 7)

In [252]:
map_Durham_venues_office = folium.Map(location=[latitude, longitude], zoom_start=11)
addToMap(Durham_venues_office , 'red', map_Durham_venues_office)
map_Durham_venues_office 

In [253]:
Durham_venues_shop = getNearbyVenues(names=df3['Durham'], latitudes=df3['Latitude'], longitudes=df3['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1fd941735')
Durham_venues_shop

,Durham,Durham Latitude,Durham Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Ajax, Ontario",43.850374,-79.024658,riocan,43.857014,-79.022964,Shopping Mall
1,"Pickering, Ontario",43.835765,-79.090576,Pickering Town Centre,43.836032,-79.086304,Shopping Mall
2,"Pickering, Ontario",43.835765,-79.090576,Pickering Town Centre Guest Services,43.835129,-79.086270,Shopping Mall
3,"Pickering, Ontario",43.835765,-79.090576,Santa,43.835392,-79.086043,Shopping Mall
4,"Pickering, Ontario",43.835765,-79.090576,The Hub,43.838132,-79.089718,Shopping Mall
5,"Pickering, Ontario",43.835765,-79.090576,Pickering Town Centre Door 7,43.834898,-79.086397,Shopping Mall
6,"Pickering, Ontario",43.835765,-79.090576,Epic Hair Salon,43.835942,-79.086426,Shopping Mall


In [254]:
Durham_venues_shop.shape

(7, 7)

In [257]:
map_Durham_venues_shop= folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(Durham_venues_shop, 'purple', map_Durham_venues_shop)
map_Durham_venues_shop

In [258]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Durham').count()
    
    for n in startDf['Durham']:
        try:
            startDf.loc[startDf['Durham'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Durham'] == n,columnTitle] = 0

In [264]:
df_data = df3.copy()
df_data.rename(columns={'Durham':'Durham'}, inplace=True)
addColumn(df_data, 'Fast Food Restaurant', Durham_venues_fastfood)
addColumn(df_data, 'High Schools',Durham_venues_highschools)
addColumn(df_data, 'Universities', Durham_venues_uni)
addColumn(df_data, 'Malls', Durham_venues_shop)
addColumn(df_data, 'Offices', Durham_venues_office)
df_data

,Durham,Latitude,Longitude,Fast Food Restaurant,High Schools,Universities,Malls,Offices
0,"Whitby, Ontario",43.899871,-78.940201,2.0,4.0,2.0,0.0,39.0
1,"Ajax, Ontario",43.850374,-79.024658,13.0,7.0,4.0,1.0,39.0
2,Clarington,43.968686,-78.651357,0.0,0.0,0.0,0.0,0.0
3,"Pickering, Ontario",43.835765,-79.090576,17.0,10.0,6.0,6.0,44.0
4,Scugog,44.115055,-78.905989,0.0,0.0,0.0,0.0,1.0
5,"Uxbridge, Ontario",44.107556,-79.127655,0.0,6.0,1.0,0.0,33.0
6,"Brock, Ontario",44.350079,-79.101226,0.0,0.0,0.0,0.0,0.0


In [265]:
# negative weight, because Jeronimo wants to open a burger joint and thus wants to avoid concurrence as much as possible
weight_restaurant = -1

# positive weight, because high school students are good customers
weight_schools = 1

# positive weight, because uni students are good customers
weight_uni = 1.5

# positive weight because employees are even better customers
weight_office = 2

# positive weight because shopping customers are even better customers
weight_mall=3

In [266]:
df_weighted = df_data[['Durham']].copy()

In [268]:
df_weighted['Score'] = df_data['Fast Food Restaurant'] * weight_restaurant + df_data['High Schools'] * weight_schools + df_data['Universities'] * weight_uni + df_data['Malls'] * weight_malls* weight_office + df_data['Offices'] 
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,Durham,Score
3,"Pickering, Ontario",70.0
0,"Whitby, Ontario",44.0
1,"Ajax, Ontario",43.0
5,"Uxbridge, Ontario",40.5
4,Scugog,1.0
2,Clarington,0.0
6,"Brock, Ontario",0.0


In [296]:
map_Durham_result = folium.Map(location=[latitude, longitude], zoom_start=10)
folium.Marker([43.835765, -79.090576], popup='<i>Pickering, Ontario</i>').add_to(map_Durham_result)
Durham_win = df3[df3['Durham'] == 'Pickering, Ontario']

for lat, lng, local in zip(Durham_win['Latitude'], Durham_win['Longitude'], Durham_win['Durham']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.9).add_to(map_Durham_result) 

addToMap(Durham_venues_fastfood[Durham_venues_fastfood['Durham'] == 'Pickering, Ontario'], 'red', map_Durham_result)
addToMap(Durham_venues_highschools[Durham_venues_highschools['Durham'] == 'Pickering, Ontario'], 'green', map_Durham_result)
addToMap(Durham_venues_uni[Durham_venues_uni['Durham'] == 'Pickering, Ontario'], 'gold', map_Durham_result)
addToMap(Durham_venues_shop[Durham_venues_shop['Durham'] == 'Pickering, Ontario'], 'fuchsia', map_Durham_result)
addToMap(Durham_venues_office[Durham_venues_office['Durham'] == 'Pickering, Ontario'], 'fuchsia', map_Durham_result)

map_Durham_result

### Pickering Locality is the best option for Ravi in order to open his Fast Food Restaurant